In [1]:
# !pip install -qU python-gdcm pydicom pylibjpeg
# !pip install -U pylibjpeg-libjpeg -v
# !pip install pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg
# !pip install pydicom

# import include nesscessaryz
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil

## method origin

# Processing data RSNA

In [175]:
from Setting_data import *

In [4]:
meta_rsna = pd.read_csv(r"D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Collect_Data\rsna.csv")

In [5]:
meta_rsna

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54701,1,9973,1729524723,R,MLO,43.0,0,0,0,1.0,0,C,49,False
54702,1,9989,63473691,L,MLO,60.0,0,0,0,NaN,0,C,216,False
54703,1,9989,1078943060,L,CC,60.0,0,0,0,NaN,0,C,216,False
54704,1,9989,398038886,R,MLO,60.0,0,0,0,0.0,0,C,216,True


In [6]:
sd1 = Setting_data(meta_rsna)

In [7]:
meta_rsna.patient_id.unique()

array([10006, 10011, 10025, ...,   997,  9973,  9989], dtype=int64)

In [8]:
sd1.multi_process(meta_rsna.patient_id.unique())

d:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Setting_data.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['id_img_zip'] = None
d:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Setting_data.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['id_img_zip'][i] = f'{self.data.cancer[i]}_{self.data.image_id[i]}'


In [9]:
sd1.meta_pro[sd1.meta_pro.patient_id == 65222]

,patient_id,image_id,view,laterality,age,cancer
46996,65222,358676381,CC,R,61.0,0
46997,65222,1581943313,MLO,R,61.0,0
46998,65222,1559227517,CC,L,61.0,1
46999,65222,749095883,MLO,L,61.0,1
47000,65222,111055050,MLO,L,61.0,1
47001,65222,111055050,CC,L,61.0,0


In [ ]:
sd1.save_file('Test_data/rsna_7.csv')

## Đánh giá số lượng CC và MLO

### class Evaluate

In [3]:
class Evaluate:
    def __init__(self, meta_pro, name):
        self.al_data = meta_pro.reset_index()
        self.name = name
    def quatity_cc_mlo(self):
        print("So luong CC và MLO")
        print(self.al_data.view.value_counts())
    def lelf_right(self):
        print("So luong L và R")
        print(self.al_data.groupby(['view', 'laterality']).size())
    def show_one(self, par):
        print(self.al_data[self.al_data.patient_id == par])
    def show(self):
        ls = len(self.al_data)
        par = len(self.al_data.patient_id.unique())
        print(f'Dataset {self.name} \n Số lượng image: {ls} \n Số luong patient: {par}')
        self.quatity_cc_mlo()
        self.lelf_right()
        

In [2]:
sd1.meta_pro

NameError: name 'sd1' is not defined

In [88]:
ev1 =  Evaluate(sd1.meta_pro, 'RSNA')
ev1.show()

Dataset RSNA 
 Số lượng image: 0 
 Số luong patient: 0
So luong CC và MLO
Series([], Name: view, dtype: int64)
So luong L và R trong {}
Series([], dtype: int64)


In [446]:
al_data.view.value_counts()

CC     25088
MLO    25088
Name: view, dtype: int64

In [448]:
al_data

,index,patient_id,image_id,view,laterality,age,cancer
0,0,10006,1874946579,CC,R,61.0,0
1,1,10006,1864590858,MLO,R,61.0,0
2,2,10006,462822612,CC,L,61.0,0
3,3,10006,1459541791,MLO,L,61.0,0
4,4,10011,541722628,CC,R,55.0,0
...,...,...,...,...,...,...,...
50171,50171,9973,1703611570,MLO,L,43.0,0
50172,50172,9989,439796429,CC,R,60.0,0
50173,50173,9989,398038886,MLO,R,60.0,0
50174,50174,9989,1078943060,CC,L,60.0,0


In [449]:
al_data.groupby(['view', 'laterality']).size()

view  laterality
CC    L             12534
      R             12554
MLO   L             12534
      R             12554
dtype: int64

# Số liệu khá ổn


# Vindr

In [2]:
data_finding = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Collect_Data\Vindr\finding_annotations.csv')
data_finding.columns


Index(['study_id', 'series_id', 'image_id', 'laterality', 'view_position',
       'height', 'width', 'breast_birads', 'breast_density',
       'finding_categories', 'finding_birads', 'xmin', 'ymin', 'xmax', 'ymax',
       'split'],
      dtype='object')

In [3]:
data_breast = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Collect_Data\Vindr\breast-level_annotations.csv')
data_breast.columns

Index(['study_id', 'series_id', 'image_id', 'laterality', 'view_position',
       'height', 'width', 'breast_birads', 'breast_density', 'split'],
      dtype='object')

In [4]:
data_meta = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Collect_Data\Vindr\metadata.csv')
data_meta.columns

Index(['SOP Instance UID', 'Series Instance UID', 'SOP Instance UID.1',
       'Patient's Age', 'View Position', 'Image Laterality',
       'Photometric Interpretation', 'Rows', 'Columns', 'Imager Pixel Spacing',
       'Pixel Spacing', 'Pixel Padding Value', 'Pixel Padding Range Limit',
       'Window Center', 'Window Width', 'Rescale Intercept', 'Rescale Slope',
       'Rescale Type', 'Window Center & Width Explanation', 'Manufacturer',
       'Manufacturer's Model Name'],
      dtype='object')

In [5]:
age_dict = {}
for i in range(len(data_meta['SOP Instance UID'])):
    age_dict[data_meta['SOP Instance UID'][i]] = float(str(data_meta["Patient's Age"][i]).replace('Y', ''))

In [183]:
len(data_finding)

20486

In [6]:
meta_vindr = data_breast[['study_id' , 'image_id', 'view_position','laterality','breast_birads']]

In [7]:
ls_age = []

In [8]:
for i in range(len(meta_vindr['breast_birads'])):
    # meta_vindr['age'][i] = age_dict[meta_vindr['study_id'][i]]
    if age_dict[meta_vindr['image_id'][i]] == None:
        print(meta_vindr.image_id[i])
    ls_age.append(age_dict[meta_vindr['image_id'][i]])
    # print(meta_vindr['breast_birads'][i])
    if meta_vindr['breast_birads'][i] == 'BI-RADS 5':
        
        meta_vindr['breast_birads'][i] = 1
    else:
        meta_vindr['breast_birads'][i] = 0
meta_vindr['age'] = ls_age

C:\Users\nguye\AppData\Local\Temp\ipykernel_7784\1393809994.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_vindr['breast_birads'][i] = 0
C:\Users\nguye\AppData\Local\Temp\ipykernel_7784\1393809994.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_vindr['breast_birads'][i] = 1
C:\Users\nguye\AppData\Local\Temp\ipykernel_7784\1393809994.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [187]:
meta_vindr = meta_vindr.rename(columns={'study_id':'patient_id', 'image_id': 'image_id', 'view_position': 'view', 'laterality': 'laterality', 'breast_birads': 'cancer'})

In [188]:
meta_vindr = meta_vindr[['patient_id', 'image_id', 'view', 'laterality', 'age', 'cancer']]

In [189]:
# Lấy trung bình tuổi của cancer
avg_age = int(np.sum(meta_vindr[meta_vindr.cancer == 1].age) / len(meta_vindr[meta_vindr.cancer == 1]))
avg_age

42

In [190]:
meta_vindr.age.fillna(avg_age, inplace=True)

In [191]:
meta_vindr[meta_vindr.image_id == "37858478f893eb85f25e577538167e75"]

,patient_id,image_id,view,laterality,age,cancer
15668,ae4b6f49514c32216a8aabb4b3ad026a,37858478f893eb85f25e577538167e75,MLO,L,42.0,0


+ Nhận xét bộ dữ liệu vindr mỗi bệnh nhân chỉ có 4 tấm

In [192]:
sd1 = Setting_data(meta_vindr)
# data_group, data = sd1.zip_idimg_cancer()
# sd1.procee_par('ae4b6f49514c32216a8aabb4b3ad026a')
sd1.multi_process(meta_vindr.patient_id.unique())

# sd1.save_file('Test_data/vindr_1.csv')

d:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Setting_data.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['id_img_zip'][i] = f'{self.data.cancer[i]}_{self.data.image_id[i]}'


In [194]:
sd1.save_file('Test_data/vindr_8.csv')

In [195]:
ds = sd1.meta_pro[sd1.meta_pro.patient_id == '15a411e94e743361a5725ccf6b728ded']
ds

,patient_id,image_id,view,laterality,age,cancer
16938,15a411e94e743361a5725ccf6b728ded,fc869569babcc087c18d01ff23b27a18,CC,R,42.0,1
16939,15a411e94e743361a5725ccf6b728ded,e8807d2cd8bceca2378375fb986cf578,MLO,R,42.0,1
16940,15a411e94e743361a5725ccf6b728ded,7c6e84f9f6d99bc20b1c2a69af32bcc7,CC,L,42.0,0
16941,15a411e94e743361a5725ccf6b728ded,7f50dbf9a99c1372d0ec335f03a1037e,MLO,L,42.0,0


In [96]:
ds.groupby(['patient_id', 'view', 'laterality', 'id_img_zip', 'age']).size()

Series([], dtype: int64)

In [83]:
len(data.index)

17740

In [91]:
# data.index[i][0]
for i in range(17740):
    # print(i)
    if data.index[i][0]  == 'ae4b6f49514c32216a8aabb4b3ad026a': print("YES")

In [63]:
sd1.meta_pro

,patient_id,image_id,view,laterality,age,cancer


In [36]:
origin = meta_vindr[meta_vindr.cancer == 1].image_id
origin

66       581b0e4d471a5c0adc6888cb038fa722
67       29896919b15b2dd978a36b2f68a5ad93
456      5d13674e2ed04208bf232e2eeb468f7a
457      d4cae2c614676925578ca87836ab95b6
1060     39f3ef0e8046a9b4e6c81262d634053d
                       ...               
19697    19acc4b912b5637af651392bc1fe6b6e
19702    19269b87da23b12053b3bfcfa65a2eeb
19703    42f76304c7825a09405739d899c6cd86
19870    85a6579cbdc403cfc4dde0a8149ed855
19871    10e0f362333df810ac84a9db8fb3fd42
Name: image_id, Length: 226, dtype: object

In [40]:
new = list(sd1.meta_pro[sd1.meta_pro.cancer == 1].image_id)
new

['581b0e4d471a5c0adc6888cb038fa722',
 '29896919b15b2dd978a36b2f68a5ad93',
 '5d13674e2ed04208bf232e2eeb468f7a',
 'd4cae2c614676925578ca87836ab95b6',
 '39f3ef0e8046a9b4e6c81262d634053d',
 'cb02caed6492a9619e93a3bd265a9c7f',
 'e61d36901ebb6b37ac41f6ca84e5c81b',
 'b74f95f32636aff0b94540891c6b1f14',
 '271e8bfd46adee65f8580092753d08e5',
 '3c315903764b853c05c059346675e1b7',
 '73d9a9cb6d8d2685271440b9bd0bfc4b',
 'fde50065b6d598d0bb62a08a0f0d97b7',
 '6ffb4582bf7434518000863d13bf8915',
 '27d9bf4e5b772624325d6232ffc8378a',
 '153e6a233a6694e96db4356a947c9678',
 '733a9b17716849f4fac702227c0414fa',
 '96eb9c4bdf059f2f8da825bfc0a3ba69',
 '53c092e62d94e3193734c7bdf81a6b49',
 '5fe8350c930a08ebc532cbb304f5642a',
 '7a6c57ce6034053ca02a8cd32549db7f',
 '852092131d24851ff0e927b68cce2db7',
 'fb16653152c4808c9e759888d15e42c4',
 '37a23b32dea999c01e4bc46901b21d67',
 '93960b56ba5808e9047aff73cf9b0762',
 '225863035d1c03f3c1cd49f409870fde',
 'e751e5a947089704457741eea74c2280',
 '6c89a90416411eecf56e5ea1fff2a1e2',
 

In [46]:
for id in origin:
    if id in new:
        # print(id)
        id = id
    else:
        print(meta_vindr[meta_vindr.image_id == id].patient_id.values)

['ae4b6f49514c32216a8aabb4b3ad026a']
['ae4b6f49514c32216a8aabb4b3ad026a']
['d539fcf54d3a20362f11a47d5b2e0cd0']
['d539fcf54d3a20362f11a47d5b2e0cd0']
['9bb21300e3a92937024437cacc180158']
['9bb21300e3a92937024437cacc180158']
['e6a3e149e9af887c6f26074601676a3f']
['e6a3e149e9af887c6f26074601676a3f']
['15a411e94e743361a5725ccf6b728ded']
['15a411e94e743361a5725ccf6b728ded']
['7ed7391fe55395a5e33c4254788c1058']
['7ed7391fe55395a5e33c4254788c1058']
['bf90ae111b0e4727bc583fe31114b1d6']
['bf90ae111b0e4727bc583fe31114b1d6']
['0f1a7b0d4efecb388a4b98ae5e2d5c29']
['0f1a7b0d4efecb388a4b98ae5e2d5c29']


In [47]:
meta_vindr[meta_vindr.patient_id == 'ae4b6f49514c32216a8aabb4b3ad026a']

,patient_id,image_id,view,laterality,age,cancer
15668,ae4b6f49514c32216a8aabb4b3ad026a,37858478f893eb85f25e577538167e75,MLO,L,NaN,0
15669,ae4b6f49514c32216a8aabb4b3ad026a,716141d4aefa3ac2bdfc97f9a74dbf01,CC,L,NaN,0
15670,ae4b6f49514c32216a8aabb4b3ad026a,ac315834dc489efe6d1e96d93ae8369d,MLO,R,NaN,1
15671,ae4b6f49514c32216a8aabb4b3ad026a,16772df2fc68173354ed7b5684e40a56,CC,R,NaN,1


In [17]:
ev2 =  Evaluate(sd1.meta_pro, 'Vindr')
ev2.show()

Dataset Vindr 
 Số lượng image: 17738 
 Số luong patient: 4435
So luong CC và MLO
CC     8869
MLO    8869
Name: view, dtype: int64
So luong L và R trong {}
view  laterality
CC    L             4434
      R             4435
MLO   L             4434
      R             4435
dtype: int64


# CMMD

In [1]:
meta_cmmd = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Collect_Data\Cmmd_external-1 (1).csv')

NameError: name 'pd' is not defined

In [130]:
meta_cmmd = meta_cmmd.rename(columns={'patient_id':'patient_id', 'img_id': 'image_id', 'view': 'view', 'laterality': 'laterality', 'cancer': 'cancer'})

In [131]:
meta_cmmd = meta_cmmd[['path_img', 'patient_id', 'image_id', 'view', 'laterality', 'age', 'cancer']]
meta_cmmd.image_id = meta_cmmd.image_id.apply(lambda x:  x.replace('_', '&'))

In [132]:
meta_cmmd

,patient_id,image_id,view,laterality,age,cancer
0,D1-0001,D1-0001&1-1,MLO,R,44,0
1,D1-0001,D1-0001&1-2,CC,R,44,0
2,D1-0002,D1-0002&1-1,MLO,L,40,0
3,D1-0002,D1-0002&1-2,CC,L,40,0
4,D1-0003,D1-0003&1-2,CC,L,39,0
...,...,...,...,...,...,...
5585,D2-0748,D2-0748&1-2,CC,R,29,0
5586,D2-0749,D2-0749&1-2,CC,L,42,1
5587,D2-0749,D2-0749&1-4,CC,L,42,0
5588,D2-0749,D2-0749&1-3,MLO,L,42,0


In [33]:
sd3 = Setting_data(meta_cmmd)

sd3.multi_process(meta_cmmd.patient_id.unique())

sd3.save_file('Test_data/cmmd_1.csv')

d:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Setting_data.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['id_img_zip'][i] = f'{self.data.cancer[i]}_{self.data.image_id[i]}'


In [34]:
ev3 =  Evaluate(sd3.meta_pro, 'Cmmd')
ev3.show()

Dataset Cmmd 
 Số lượng image: 5590 
 Số luong patient: 1775
So luong CC và MLO
CC     2795
MLO    2795
Name: view, dtype: int64
So luong L và R trong {}
view  laterality
CC    L             1461
      R             1334
MLO   L             1461
      R             1334
dtype: int64


# CDD_CESM

In [39]:
meta_cdd = pd.read_csv(r"D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Collect_Data\CDD_CESM_external.csv")
meta_cdd

,Unnamed: 0,image_id,patient_id,laterality,view,age,machine_id,cancer
0,0,P1_L_DM_MLO,1,L,MLO,46,1,1
1,1,P2_R_DM_CC,2,R,CC,31,2,0
2,2,P2_R_DM_MLO,2,R,MLO,31,2,0
3,3,P2_L_DM_CC,2,L,CC,31,2,0
4,4,P2_L_DM_MLO,2,L,MLO,31,2,0
...,...,...,...,...,...,...,...,...
998,998,P325_L_DM_MLO,325,L,MLO,49,1,0
999,999,P326_R_DM_CC,326,R,CC,51,1,0
1000,1000,P326_R_DM_MLO,326,R,MLO,51,1,0
1001,1001,P326_L_DM_CC,326,L,CC,51,1,1


In [40]:
meta_cdd = meta_cdd[['patient_id', 'image_id', 'view', 'laterality', 'age', 'cancer']]
meta_cdd.image_id = meta_cdd.image_id.apply(lambda x:  x.replace('_', '&'))

C:\Users\nguye\AppData\Local\Temp\ipykernel_7784\459961279.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_cdd.image_id = meta_cdd.image_id.apply(lambda x:  x.replace('_', '&'))


In [41]:
meta_cdd

,patient_id,image_id,view,laterality,age,cancer
0,1,P1&L&DM&MLO,MLO,L,46,1
1,2,P2&R&DM&CC,CC,R,31,0
2,2,P2&R&DM&MLO,MLO,R,31,0
3,2,P2&L&DM&CC,CC,L,31,0
4,2,P2&L&DM&MLO,MLO,L,31,0
...,...,...,...,...,...,...
998,325,P325&L&DM&MLO,MLO,L,49,0
999,326,P326&R&DM&CC,CC,R,51,0
1000,326,P326&R&DM&MLO,MLO,R,51,0
1001,326,P326&L&DM&CC,CC,L,51,1


In [44]:
from Setting_data import *

In [45]:
sd4 = Setting_data(meta_cdd)

sd4.multi_process(meta_cdd.patient_id.unique())

# sd4.save_file('Test_data/cdd_cesm.csv')

d:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Setting_data.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['id_img_zip'][i] = f'{self.data.cancer[i]}_{self.data.image_id[i]}'


[('0_P2&R&DM&CC', 31)]
[('0_P2&R&DM&MLO', 31)]
P2&R&DM&CC P2&R&DM&MLO
[('0_P2&L&DM&CC', 31)]
[('0_P2&L&DM&MLO', 31)]
P2&L&DM&CC P2&L&DM&MLO
[('0_P3&R&DM&CC', 45)]
[('0_P3&R&DM&MLO', 45)]
P3&R&DM&CC P3&R&DM&MLO
[('1_P3&L&DM&CC', 45)]
[('1_P3&L&DM&MLO', 45)]
P3&L&DM&CC P3&L&DM&MLO
[('0_P5&R&DM&CC', 51)]
[('0_P5&R&DM&MLO', 51)]
P5&R&DM&CC P5&R&DM&MLO
[('0_P5&L&DM&CC', 51)]
[('0_P5&L&DM&MLO', 51)]
P5&L&DM&CC P5&L&DM&MLO
[('0_P7&R&DM&CC', 69)]
[('0_P7&R&DM&MLO', 69)]
P7&R&DM&CC P7&R&DM&MLO
[('0_P10&R&DM&CC ', 63)]
[('0_P10&R&DM&MLO ', 63)]
P10&R&DM&CC  P10&R&DM&MLO 
[('0_P10&L&DM&CC', 63)]
[('0_P10&L&DM&MLO', 63)]
P10&L&DM&CC P10&L&DM&MLO
[('0_P11&R&DM&CC', 51)]
[('0_P11&R&DM&MLO', 51)]
P11&R&DM&CC P11&R&DM&MLO
[('0_P12&R&DM&CC', 63)]
[('0_P12&R&DM&MLO', 63)]
P12&R&DM&CC P12&R&DM&MLO
[('1_P12&L&DM&CC', 63)]
[('1_P12&L&DM&MLO', 63)]
P12&L&DM&CC P12&L&DM&MLO
[('0_P14&R&DM&CC', 56)]
[('0_P14&R&DM&MLO', 56)]
P14&R&DM&CC P14&R&DM&MLO
[('0_P14&L&DM&CC', 56)]
[('0_P14&L&DM&MLO', 56)]
P14&L&DM&CC P

In [46]:
def loc_cancer(data, name):
    df = data[data['cancer'] == 1].patient_id
    df = df.unique()
    data = data[data['patient_id'].isin(df)]
    data = data.reset_index()
    data.to_csv(f'Test_data/{name}.csv')


In [47]:
loc_cancer(sd4.meta_pro, 'cdd_cesm_3')

In [61]:
loc_cancer(sd3.meta_pro, 'Cmmd_2')

In [62]:
df = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Test_data\vindr_1.csv')
df

,Unnamed: 0,patient_id,image_id,view,laterality,age,cancer
0,0,b8d273e8601f348d3664778dae0e7e0b,cd0fc7bc53ac632a11643ac4cc91002a,CC,R,53.0,0
1,1,b8d273e8601f348d3664778dae0e7e0b,71638b1e853799f227492bfb08a01491,MLO,R,53.0,0
2,2,b8d273e8601f348d3664778dae0e7e0b,d8125545210c08e1b1793a5af6458ee2,CC,L,53.0,0
3,3,b8d273e8601f348d3664778dae0e7e0b,290c658f4e75a3f83ec78a847414297c,MLO,L,53.0,0
4,4,8269f5971eaca3e5d3772d1796e6bd7a,202d761a6b0f86faaeefc39ee18b1c53,CC,R,42.0,0
...,...,...,...,...,...,...,...
17733,17733,f2093a752e6b44df5990f5fd38c99dd2,9edcc7df8ac157fb2fdbecb9baae9f2f,MLO,L,38.0,0
17734,17734,b3c8969cd2accfa4dbb2aece1f7158ab,4689616c3d0b46fcba7a771107730791,CC,R,18.0,0
17735,17735,b3c8969cd2accfa4dbb2aece1f7158ab,d443b9725e331b8b27589aa725597801,MLO,R,18.0,0
17736,17736,b3c8969cd2accfa4dbb2aece1f7158ab,3c22491bcf1d0b004715c28d80981cdd,CC,L,18.0,0


In [ ]:
loc_cancer(df, 'Vindr_2')

## Check last time

In [48]:
class Evaluate:
    def __init__(self, meta_pro, name):
        self.al_data = meta_pro.reset_index()
        self.name = name
    def quatity_cc_mlo(self):
        print("So luong CC và MLO")
        print(self.al_data.view.value_counts())
    def lelf_right(self):
        print("So luong L và R")
        print(self.al_data.groupby(['view', 'laterality']).size())
    def show_one(self, par):
        print(self.al_data[self.al_data.patient_id == par])
    def quanlity_cancer(self):
        sl = len(self.al_data[self.al_data.cancer == 1])
        print(f"So luong img cancer: {sl}")
        # print(self.al_data[self.al_data.cancer == 1])
    def show(self):
        ls = len(self.al_data)
        par = len(self.al_data.patient_id.unique())
        print(f'Dataset {self.name} \n Số lượng image: {ls} \n Số luong patient: {par}')
        self.quatity_cc_mlo()
        self.lelf_right()
        self.quanlity_cancer()

RSNA

In [146]:
rsna = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Test_data\rsna_7.csv')
rsna

,Unnamed: 0,patient_id,image_id,view,laterality,age,cancer
0,0,10006,1874946579,CC,R,61.0,0
1,1,10006,1864590858,MLO,R,61.0,0
2,2,10006,462822612,CC,L,61.0,0
3,3,10006,1459541791,MLO,L,61.0,0
4,4,10011,541722628,CC,R,55.0,0
...,...,...,...,...,...,...,...
50171,50171,9973,1703611570,MLO,L,43.0,0
50172,50172,9989,439796429,CC,R,60.0,0
50173,50173,9989,398038886,MLO,R,60.0,0
50174,50174,9989,1078943060,CC,L,60.0,0


In [147]:
rsna = rsna[['patient_id', 'image_id', 'view', 'laterality', 'age', 'cancer']]
rsna

,patient_id,image_id,view,laterality,age,cancer
0,10006,1874946579,CC,R,61.0,0
1,10006,1864590858,MLO,R,61.0,0
2,10006,462822612,CC,L,61.0,0
3,10006,1459541791,MLO,L,61.0,0
4,10011,541722628,CC,R,55.0,0
...,...,...,...,...,...,...
50171,9973,1703611570,MLO,L,43.0,0
50172,9989,439796429,CC,R,60.0,0
50173,9989,398038886,MLO,R,60.0,0
50174,9989,1078943060,CC,L,60.0,0


In [148]:
ev_rsna = Evaluate(rsna, 'rsna')
ev_rsna.show()

Dataset rsna 
 Số lượng image: 50176 
 Số luong patient: 11905
So luong CC và MLO
CC     25088
MLO    25088
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             12534
      R             12554
MLO   L             12534
      R             12554
dtype: int64
So luong img cancer: 1156


### Vindr

In [12]:
vindr = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Test_data\Vindr_8.csv')
vindr = vindr[['patient_id', 'image_id', 'view', 'laterality', 'age', 'cancer']]
cancer = list(vindr[vindr.cancer == 1].patient_id.values)
cancer

['10200894bf6f74b2ae92a0cecd37c60c',
 '10200894bf6f74b2ae92a0cecd37c60c',
 '450f7f9b2aa32754b076f0f53f0d8efc',
 '450f7f9b2aa32754b076f0f53f0d8efc',
 'fe23c1647f7617ef219a0a0e07c9eec5',
 'fe23c1647f7617ef219a0a0e07c9eec5',
 'a0da8d047c71765cdf217a8a52606741',
 'a0da8d047c71765cdf217a8a52606741',
 'a0d75cb5a7be01a0d63f5d9ec553443e',
 'a0d75cb5a7be01a0d63f5d9ec553443e',
 '563643e06302bcd7216c2ca8de3157e5',
 '563643e06302bcd7216c2ca8de3157e5',
 'de585e55834681c650d1f34ff21c2139',
 'de585e55834681c650d1f34ff21c2139',
 '877bde1cb8b7df49cde3fdb7020a1536',
 '877bde1cb8b7df49cde3fdb7020a1536',
 'd8d0bc6e58b11efa2f86a9c0986b038e',
 'd8d0bc6e58b11efa2f86a9c0986b038e',
 'b57b97020df361371ae1877080433275',
 'b57b97020df361371ae1877080433275',
 'ad39689f1bea2e0aee2574488f06d231',
 'ad39689f1bea2e0aee2574488f06d231',
 'f637a8ea804bac3941603ae34dae7fb7',
 'f637a8ea804bac3941603ae34dae7fb7',
 '1560345de7d8277b170ca3842eae6c97',
 '1560345de7d8277b170ca3842eae6c97',
 '8dd986e8174fc84c2a984e66a416ff9f',
 

In [13]:
vindr = vindr[vindr.patient_id.isin(cancer)]

In [16]:
ev_vin = Evaluate(vindr, 'vindr')
ev_vin.show()

Dataset vindr 
 Số lượng image: 452 
 Số luong patient: 113
So luong CC và MLO
CC     226
MLO    226
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             113
      R             113
MLO   L             113
      R             113
dtype: int64
So luong img cancer: 226


In [18]:
vindr = vindr.reset_index()
vindr.to_csv('Test_data/vindr_9.csv')

### CMMD

In [35]:
cmmd = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Test_data\Cmmd_2.csv')
cmmd = cmmd[['patient_id', 'image_id', 'view', 'laterality', 'age', 'cancer']]
cancer_cmmd = list(cmmd[cmmd.cancer == 1].patient_id.unique())
cancer_cmmd

['D1-0045',
 'D1-0046',
 'D1-0047',
 'D1-0051',
 'D1-0053',
 'D1-0055',
 'D1-0056',
 'D1-0058',
 'D1-0059',
 'D1-0061',
 'D1-0063',
 'D1-0064',
 'D1-0067',
 'D1-0068',
 'D1-0070',
 'D1-0071',
 'D1-0075',
 'D1-0078',
 'D1-0084',
 'D1-0085',
 'D1-0087',
 'D1-0089',
 'D1-0091',
 'D1-0101',
 'D1-0102',
 'D1-0103',
 'D1-0105',
 'D1-0106',
 'D1-0107',
 'D1-0110',
 'D1-0111',
 'D1-0112',
 'D1-0113',
 'D1-0116',
 'D1-0120',
 'D1-0121',
 'D1-0122',
 'D1-0124',
 'D1-0127',
 'D1-0129',
 'D1-0130',
 'D1-0131',
 'D1-0132',
 'D1-0133',
 'D1-0134',
 'D1-0135',
 'D1-0140',
 'D1-0142',
 'D1-0143',
 'D1-0147',
 'D1-0148',
 'D1-0149',
 'D1-0152',
 'D1-0154',
 'D1-0157',
 'D1-0161',
 'D1-0162',
 'D1-0163',
 'D1-0168',
 'D1-0169',
 'D1-0171',
 'D1-0173',
 'D1-0176',
 'D1-0178',
 'D1-0180',
 'D1-0181',
 'D1-0182',
 'D1-0183',
 'D1-0184',
 'D1-0188',
 'D1-0189',
 'D1-0191',
 'D1-0192',
 'D1-0194',
 'D1-0196',
 'D1-0199',
 'D1-0200',
 'D1-0201',
 'D1-0202',
 'D1-0203',
 'D1-0204',
 'D1-0207',
 'D1-0208',
 'D1

In [38]:
cmmd

,patient_id,image_id,view,laterality,age,cancer
0,D1-0045,D1-0045&1-2,CC,L,33,1
1,D1-0045,D1-0045&1-1,MLO,L,33,1
2,D1-0046,D1-0046&1-2,CC,R,43,1
3,D1-0046,D1-0046&1-1,MLO,R,43,1
4,D1-0047,D1-0047&1-2,CC,R,43,1
...,...,...,...,...,...,...
4289,D2-0748,D2-0748&1-1,MLO,R,29,0
4290,D2-0749,D2-0749&1-2,CC,L,42,1
4291,D2-0749,D2-0749&1-1,MLO,L,42,1
4292,D2-0749,D2-0749&1-4,CC,L,42,0


In [37]:
cmmd.patient_id.unique()

array(['D1-0045', 'D1-0046', 'D1-0047', ..., 'D2-0747', 'D2-0748',
       'D2-0749'], dtype=object)

In [34]:
len(cmmd.patient_id.unique())

1310

In [36]:
len(cancer_cmmd)

1310

In [26]:
cmmd = cmmd[cmmd.patient_id.isin(cancer_cmmd)]

In [27]:
ev_cmmd = Evaluate(cmmd, 'CMMD')
ev_cmmd.show()

Dataset CMMD 
 Số lượng image: 4294 
 Số luong patient: 1310
So luong CC và MLO
CC     2147
MLO    2147
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             1133
      R             1014
MLO   L             1133
      R             1014
dtype: int64
So luong img cancer: 2633


### CDD-CESM

In [49]:
CDD = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Notebook_code\Test_data\cdd_cesm_3.csv')
CDD = CDD[['patient_id', 'image_id', 'view', 'laterality', 'age', 'cancer']]
CDD

,patient_id,image_id,view,laterality,age,cancer
0,1,P1&L&DM&MLO,MLO,L,46,1
1,1,P1&L&DM&MLO,CC,L,46,0
2,3,P3&R&DM&CC,CC,R,45,0
3,3,P3&R&DM&MLO,MLO,R,45,0
4,3,P3&L&DM&CC,CC,L,45,1
...,...,...,...,...,...,...
633,323,P323&L&DM&MLO,MLO,L,45,1
634,326,P326&R&DM&CC,CC,R,51,0
635,326,P326&R&DM&MLO,MLO,R,51,0
636,326,P326&L&DM&CC,CC,L,51,1


In [50]:
ev_CDD = Evaluate(CDD, 'CDD')
ev_CDD.show()

Dataset CDD 
 Số lượng image: 638 
 Số luong patient: 179
So luong CC và MLO
MLO    319
CC     319
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             164
      R             155
MLO   L             164
      R             155
dtype: int64
So luong img cancer: 331


In [111]:
cmmd.groupby(['patient_id'])['image_id'].count().value_counts()

4    140
2     39
Name: image_id, dtype: int64

In [115]:
rsna = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Collect_Data\rsna.csv')

In [116]:
rsna.groupby(['patient_id'])['image_id'].count().value_counts()

4     8233
5     1767
6     1218
7      276
8      229
9       99
10      53
11      19
12      12
13       5
14       2
Name: image_id, dtype: int64

In [119]:
meta_vindr

,study_id,image_id,view_position,laterality,breast_birads,age
0,b8d273e8601f348d3664778dae0e7e0b,d8125545210c08e1b1793a5af6458ee2,CC,L,0,53.0
1,b8d273e8601f348d3664778dae0e7e0b,290c658f4e75a3f83ec78a847414297c,MLO,L,0,53.0
2,b8d273e8601f348d3664778dae0e7e0b,cd0fc7bc53ac632a11643ac4cc91002a,CC,R,0,53.0
3,b8d273e8601f348d3664778dae0e7e0b,71638b1e853799f227492bfb08a01491,MLO,R,0,53.0
4,8269f5971eaca3e5d3772d1796e6bd7a,dd9ce3288c0773e006a294188aadba8e,CC,L,0,42.0
...,...,...,...,...,...,...
19995,f2093a752e6b44df5990f5fd38c99dd2,ea732154d149f619b20070b78060ae65,CC,R,0,38.0
19996,b3c8969cd2accfa4dbb2aece1f7158ab,4689616c3d0b46fcba7a771107730791,CC,R,0,18.0
19997,b3c8969cd2accfa4dbb2aece1f7158ab,3c22491bcf1d0b004715c28d80981cdd,CC,L,0,18.0
19998,b3c8969cd2accfa4dbb2aece1f7158ab,d443b9725e331b8b27589aa725597801,MLO,R,0,18.0


In [128]:
meta_vindr.groupby(['patient_id'])['image_id'].count().value_counts()

4    5000
Name: image_id, dtype: int64

In [133]:
meta_cmmd.groupby(['patient_id'])['image_id'].count().value_counts()

2    949
4    729
8     97
Name: image_id, dtype: int64

In [137]:
meta_cdd.groupby(['patient_id'])['image_id'].count().value_counts()

4    191
1     62
2     46
3     25
5      2
Name: image_id, dtype: int64